# 第四章：區塊、遮蔽與控制結構 (Blocks, Shadowing & Control Structures)

本章深入探討 Go 語言的程式流程控制機制，包括變數作用域、條件判斷、迴圈結構等核心概念。

## 學習重點
- 理解區塊作用域和變數遮蔽機制
- 掌握 if 條件判斷的各種形式
- 熟練運用四種 for 迴圈
- 學會 switch 語句的靈活應用
- 了解 break、continue、goto 的使用時機
- 掌握 for-range 的各種用法

---

## 1. 區塊 (Blocks) 與作用域

區塊是用大括號 `{}` 包圍的程式碼區域，定義了變數的作用域。Go 語言有以下幾種區塊類型：

### 區塊的類型
- **宇宙區塊 (Universe Block)**: 包含所有 Go 原始碼
- **套件區塊 (Package Block)**: 單一套件中的所有程式碼
- **檔案區塊 (File Block)**: 單一檔案中的程式碼
- **函數區塊**: 函數體內的程式碼
- **明確區塊**: 用 `{}` 明確標示的區塊

In [ ]:
//go:build ignore
package main

import "fmt"

// 套件層級變數
var packageVar = "我是套件變數"

func main() {
    // 函數層級變數
    funcVar := "我是函數變數"
    fmt.Printf("套件變數: %s\n", packageVar)
    fmt.Printf("函數變數: %s\n", funcVar)
    
    // 明確區塊
    {
        blockVar := "我是區塊變數"
        fmt.Printf("區塊內可存取: %s, %s, %s\n", packageVar, funcVar, blockVar)
        
        // 巢狀區塊
        {
            nestedVar := "我是巢狀區塊變數"
            fmt.Printf("巢狀區塊內: %s\n", nestedVar)
        }
        // fmt.Println(nestedVar) // 編譯錯誤！nestedVar 已超出作用域
    }
    // fmt.Println(blockVar) // 編譯錯誤！blockVar 已超出作用域
    
    // 控制結構也會建立區塊
    if true {
        ifVar := "if 區塊變數"
        fmt.Printf("if 區塊內: %s\n", ifVar)
    }
    // fmt.Println(ifVar) // 編譯錯誤！
}

## 2. 遮蔽變數 (Variable Shadowing)

當內層區塊宣告與外層區塊同名的變數時，會發生變數遮蔽。內層變數會"遮蔽"外層變數。

### 遮蔽的基本概念

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    // 外層變數
    x := 10
    fmt.Printf("外層 x: %d\n", x)
    
    // 第一層遮蔽
    {
        x := "hello" // 遮蔽外層的 int x
        fmt.Printf("第一層 x: %s (型別: %T)\n", x, x)
        
        // 第二層遮蔽
        {
            x := 3.14 // 遮蔽上層的 string x
            fmt.Printf("第二層 x: %.2f (型別: %T)\n", x, x)
        }
        
        fmt.Printf("回到第一層 x: %s\n", x)
    }
    
    fmt.Printf("回到外層 x: %d\n", x)
}

### 遮蔽的常見陷阱

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "strconv"
)

func demonstrateShadowingPitfalls() {
    fmt.Println("=== 常見的遮蔽陷阱 ===")
    
    // 陷阱 1: := 意外建立新變數
    var result string
    var err error
    
    if true {
        result, err := strconv.Atoi("123") // 注意：這裡用 := 建立了新的局部變數
        fmt.Printf("if 內 result: %d, err: %v\n", result, err)
    }
    
    fmt.Printf("if 外 result: %q, err: %v\n", result, err) // 外層變數未被修改
    
    // 正確做法
    if true {
        result, err = "456", nil // 使用 = 修改外層變數
    }
    
    fmt.Printf("修正後 result: %q\n", result)
}

func demonstrateImportShadowing() {
    fmt.Println("\n=== 匯入套件遮蔽 ===")
    
    // 遮蔽標準庫函數 - 不推薦！
    fmt := "我不是 fmt 套件"
    // fmt.Println("hello") // 編譯錯誤！fmt 現在是 string
    
    // 必須使用其他方式輸出
    _ = fmt // 避免未使用變數錯誤
    
    // 在區塊內恢復 fmt
    {
        // 重新匯入或使用完整路徑
        // import "fmt" // 無法在函數內重新匯入
        print("只能用 print()\n")
    }
}

func main() {
    demonstrateShadowingPitfalls()
    demonstrateImportShadowing()
}

## 3. if 條件判斷

Go 的 if 語句具有獨特的特性：
- 不需要括號包圍條件
- 支援初始化語句
- 可以搭配 else if 和 else

### if 的各種形式

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "math/rand"
    "strconv"
    "strings"
)

func main() {
    // 1. 基本 if
    age := 18
    if age >= 18 {
        fmt.Println("成年了")
    }
    
    // 2. if-else
    score := 85
    if score >= 90 {
        fmt.Println("優秀")
    } else if score >= 80 {
        fmt.Println("良好")
    } else if score >= 70 {
        fmt.Println("普通")
    } else {
        fmt.Println("需要努力")
    }
    
    // 3. 帶初始化的 if（重要特性）
    if num := rand.Intn(100); num > 50 {
        fmt.Printf("大數字: %d\n", num)
    } else {
        fmt.Printf("小數字: %d\n", num)
    }
    // fmt.Println(num) // 編譯錯誤！num 僅在 if 區塊內有效
    
    // 4. 錯誤處理的常見模式
    if value, err := strconv.Atoi("123"); err != nil {
        fmt.Printf("轉換失敗: %v\n", err)
    } else {
        fmt.Printf("轉換成功: %d\n", value)
    }
    
    // 5. 複雜條件判斷
    name := "Alice"
    email := "alice@example.com"
    
    if len(name) > 0 && strings.Contains(email, "@") {
        fmt.Println("有效的使用者資料")
    }
    
    // 6. 巢狀 if
    weather := "sunny"
    temperature := 25
    
    if weather == "sunny" {
        if temperature > 20 {
            fmt.Println("適合戶外活動")
        } else {
            fmt.Println("陽光但有點冷")
        }
    }
}

### if 語句的最佳實務

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "os"
)

// 1. 提早返回模式（避免深層巢狀）
func processUser(name, email string) error {
    // 不好的寫法：深層巢狀
    /*
    if name != "" {
        if email != "" {
            if strings.Contains(email, "@") {
                // 處理邏輯
                return nil
            } else {
                return errors.New("無效的 email 格式")
            }
        } else {
            return errors.New("email 不能為空")
        }
    } else {
        return errors.New("name 不能為空")
    }
    */
    
    // 好的寫法：提早返回
    if name == "" {
        return fmt.Errorf("name 不能為空")
    }
    
    if email == "" {
        return fmt.Errorf("email 不能為空")
    }
    
    // 處理主要邏輯
    fmt.Printf("處理使用者: %s (%s)\n", name, email)
    return nil
}

// 2. 使用 if 初始化減少作用域
func readConfig(filename string) {
    // 好的做法：限制變數作用域
    if file, err := os.Open(filename); err != nil {
        fmt.Printf("無法開啟檔案: %v\n", err)
        return
    } else {
        defer file.Close()
        fmt.Println("檔案開啟成功")
        // 處理檔案...
    }
    // file 和 err 在這裡已經超出作用域
}

// 3. 避免不必要的 else
func categorizeAge(age int) string {
    // 不好的寫法：不必要的 else
    /*
    if age < 13 {
        return "兒童"
    } else if age < 20 {
        return "青少年"
    } else if age < 60 {
        return "成人"
    } else {
        return "長者"
    }
    */
    
    // 好的寫法：清晰的條件鏈
    if age < 13 {
        return "兒童"
    }
    if age < 20 {
        return "青少年"
    }
    if age < 60 {
        return "成人"
    }
    return "長者"
}

func main() {
    // 測試函數
    err := processUser("Alice", "alice@example.com")
    if err != nil {
        fmt.Printf("錯誤: %v\n", err)
    }
    
    category := categorizeAge(25)
    fmt.Printf("年齡分類: %s\n", category)
}

## 4. for 迴圈 - 四種形式

Go 語言只有 `for` 一種迴圈，但支援四種不同的語法形式。

### 四種 for 迴圈

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "time"
)

func main() {
    fmt.Println("=== 1. 完整的 for 迴圈 ===")
    // 語法：for init; condition; post { ... }
    for i := 0; i < 5; i++ {
        fmt.Printf("i = %d\n", i)
    }
    
    fmt.Println("\n=== 2. 只有條件式的 for 迴圈（相當於 while） ===")
    // 語法：for condition { ... }
    count := 0
    for count < 3 {
        fmt.Printf("count = %d\n", count)
        count++
    }
    
    fmt.Println("\n=== 3. 無窮迴圈 ===")
    // 語法：for { ... }
    counter := 0
    for {
        if counter >= 3 {
            fmt.Println("跳出無窮迴圈")
            break
        }
        fmt.Printf("無窮迴圈 counter = %d\n", counter)
        counter++
    }
    
    fmt.Println("\n=== 4. for-range 迴圈 ===")
    // 用於遍歷集合
    slice := []string{"Go", "Python", "Java"}
    
    // 遍歷索引和值
    for index, value := range slice {
        fmt.Printf("slice[%d] = %s\n", index, value)
    }
    
    // 只遍歷值
    fmt.Println("只取值:")
    for _, value := range slice {
        fmt.Printf("值: %s\n", value)
    }
    
    // 只遍歷索引
    fmt.Println("只取索引:")
    for index := range slice {
        fmt.Printf("索引: %d\n", index)
    }
}

### for 迴圈的進階應用

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "time"
)

func main() {
    fmt.Println("=== 多變數初始化 ===")
    for i, j := 0, 10; i < j; i, j = i+1, j-1 {
        fmt.Printf("i=%d, j=%d\n", i, j)
        if i >= 3 { // 避免無窮迴圈
            break
        }
    }
    
    fmt.Println("\n=== 巢狀迴圈 ===")
    // 生成乘法表
    for i := 1; i <= 3; i++ {
        for j := 1; j <= 3; j++ {
            fmt.Printf("%d x %d = %d\t", i, j, i*j)
        }
        fmt.Println() // 換行
    }
    
    fmt.Println("\n=== 迴圈中的變數作用域 ===")
    for i := 0; i < 3; i++ {
        // 每次迭代 i 都是新的變數實例
        go func() {
            time.Sleep(100 * time.Millisecond)
            fmt.Printf("goroutine: i = %d\n", i) // 正確捕獲值
        }()
    }
    
    // 等待 goroutines 完成
    time.Sleep(200 * time.Millisecond)
    
    fmt.Println("\n=== 條件複雜的迴圈 ===")
    // 找質數
    for num := 2; num <= 20; num++ {
        isPrime := true
        for divisor := 2; divisor*divisor <= num; divisor++ {
            if num%divisor == 0 {
                isPrime = false
                break
            }
        }
        if isPrime {
            fmt.Printf("%d 是質數\n", num)
        }
    }
}

## 5. break 與 continue

- `break`: 跳出當前迴圈
- `continue`: 跳過當前迭代，繼續下一次迭代

### 基本使用

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    fmt.Println("=== break 範例 ===")
    // 找到第一個偶數就停止
    numbers := []int{1, 3, 5, 8, 9, 12, 15}
    for i, num := range numbers {
        if num%2 == 0 {
            fmt.Printf("找到第一個偶數: %d (索引: %d)\n", num, i)
            break
        }
        fmt.Printf("檢查: %d\n", num)
    }
    
    fmt.Println("\n=== continue 範例 ===")
    // 跳過偶數，只印奇數
    for i := 1; i <= 10; i++ {
        if i%2 == 0 {
            continue // 跳過偶數
        }
        fmt.Printf("奇數: %d\n", i)
    }
    
    fmt.Println("\n=== 巢狀迴圈中的 break/continue ===")
    // break 只會跳出最內層迴圈
    for i := 1; i <= 3; i++ {
        fmt.Printf("外層迴圈: i = %d\n", i)
        for j := 1; j <= 5; j++ {
            if j == 3 {
                fmt.Printf("  內層 break at j = %d\n", j)
                break // 只跳出內層迴圈
            }
            fmt.Printf("  內層: j = %d\n", j)
        }
    }
    
    fmt.Println("\n=== continue 在巢狝迴圈 ===")
    for i := 1; i <= 3; i++ {
        fmt.Printf("外層: i = %d\n", i)
        for j := 1; j <= 4; j++ {
            if j == 2 {
                fmt.Printf("  跳過 j = %d\n", j)
                continue
            }
            fmt.Printf("  內層: j = %d\n", j)
        }
    }
}

## 6. 標記語句 (Labeled Statements)

標記語句允許 `break` 和 `continue` 跳出指定的迴圈層級。

### 標記 for 迴圈

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    fmt.Println("=== 標記 break ===")
    
Outer: // 標記外層迴圈
    for i := 1; i <= 3; i++ {
        fmt.Printf("外層: i = %d\n", i)
        
        for j := 1; j <= 5; j++ {
            if i == 2 && j == 3 {
                fmt.Printf("  在 i=%d, j=%d 時跳出外層迴圈\n", i, j)
                break Outer // 跳出標記的外層迴圈
            }
            fmt.Printf("  內層: j = %d\n", j)
        }
        fmt.Printf("外層迴圈結束: i = %d\n", i)
    }
    fmt.Println("程式繼續...")
    
    fmt.Println("\n=== 標記 continue ===")
    
OuterLoop:
    for i := 1; i <= 3; i++ {
        fmt.Printf("\n外層開始: i = %d\n", i)
        
        for j := 1; j <= 4; j++ {
            if j == 2 {
                fmt.Printf("  在 j=%d 時繼續外層迴圈\n", j)
                continue OuterLoop // 繼續外層迴圈的下一次迭代
            }
            fmt.Printf("  內層: j = %d\n", j)
        }
        fmt.Printf("內層迴圈完成: i = %d\n", i) // 當 j=2 時不會執行到這裡
    }
    
    fmt.Println("\n=== 實際應用：搜尋二維陣列 ===")
    matrix := [][]int{
        {1, 2, 3},
        {4, 5, 6},
        {7, 8, 9},
    }
    
    target := 6
    found := false
    
Search:
    for i, row := range matrix {
        for j, val := range row {
            if val == target {
                fmt.Printf("找到 %d 在位置 [%d][%d]\n", target, i, j)
                found = true
                break Search // 找到後立即跳出所有迴圈
            }
        }
    }
    
    if !found {
        fmt.Printf("未找到 %d\n", target)
    }
}

## 7. for-range 詳解

`for-range` 是遍歷集合的慣用方法，支援多種資料型態。

### for-range 的各種用法

In [ ]:
//go:build ignore
package main

import "fmt"

func main() {
    fmt.Println("=== 遍歷 slice ===")
    fruits := []string{"apple", "banana", "cherry"}
    
    // 索引和值
    for i, fruit := range fruits {
        fmt.Printf("fruits[%d] = %s\n", i, fruit)
    }
    
    fmt.Println("\n=== 遍歷 array ===")
    numbers := [4]int{10, 20, 30, 40}
    for index, value := range numbers {
        fmt.Printf("numbers[%d] = %d\n", index, value)
    }
    
    fmt.Println("\n=== 遍歷 map ===")
    scores := map[string]int{
        "Alice": 95,
        "Bob":   87,
        "Carol": 92,
    }
    
    // 鍵和值
    for name, score := range scores {
        fmt.Printf("%s: %d\n", name, score)
    }
    
    // 只要鍵
    fmt.Println("只取鍵:")
    for name := range scores {
        fmt.Printf("姓名: %s\n", name)
    }
    
    fmt.Println("\n=== 遍歷字串 ===")
    text := "Hello, 世界"
    
    // 遍歷 rune（字元）
    for i, char := range text {
        fmt.Printf("位置 %d: %c (U+%04X)\n", i, char, char)
    }
    
    fmt.Println("\n=== 遍歷 channel ===")
    ch := make(chan int, 3)
    ch <- 1
    ch <- 2
    ch <- 3
    close(ch) // 必須關閉 channel
    
    for value := range ch {
        fmt.Printf("從 channel 讀取: %d\n", value)
    }
}

### for-range 的常見陷阱

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "time"
)

type Person struct {
    Name string
    Age  int
}

func main() {
    fmt.Println("=== 陷阱 1: 迴圈變數重用 ===")
    
    people := []Person{
        {"Alice", 25},
        {"Bob", 30},
        {"Carol", 35},
    }
    
    // 錯誤做法：所有 goroutine 都會看到最後一個值
    fmt.Println("錯誤做法:")
    for _, person := range people {
        go func() {
            time.Sleep(100 * time.Millisecond)
            fmt.Printf("錯誤: %s\n", person.Name) // 可能都印出 "Carol"
        }()
    }
    
    time.Sleep(200 * time.Millisecond)
    
    // 正確做法：傳遞值的副本
    fmt.Println("\n正確做法:")
    for _, person := range people {
        go func(p Person) {
            time.Sleep(100 * time.Millisecond)
            fmt.Printf("正確: %s\n", p.Name)
        }(person) // 傳遞副本
    }
    
    time.Sleep(200 * time.Millisecond)
    
    fmt.Println("\n=== 陷阱 2: 修改 slice 元素 ===")
    
    numbers := []int{1, 2, 3, 4, 5}
    
    // 錯誤：無法修改原始 slice
    fmt.Println("嘗試修改值 (無效):")
    for _, num := range numbers {
        num = num * 2 // 只修改副本
    }
    fmt.Printf("修改後: %v\n", numbers) // 未改變
    
    // 正確：使用索引修改
    fmt.Println("\n使用索引修改:")
    for i := range numbers {
        numbers[i] = numbers[i] * 2
    }
    fmt.Printf("修改後: %v\n", numbers)
    
    fmt.Println("\n=== 陷阱 3: 遍歷過程中修改 slice ===")
    
    data := []int{1, 2, 3}
    fmt.Println("原始 data:", data)
    
    // 在遍歷中修改 slice
    for i, v := range data {
        fmt.Printf("遍歷: i=%d, v=%d\n", i, v)
        if i == 1 {
            data = append(data, 4, 5) // 修改 slice
            fmt.Printf("  修改後 data: %v\n", data)
        }
    }
    // 注意：新添加的元素不會被遍歷到
    fmt.Printf("遍歷結束，最終 data: %v\n", data)
    
    fmt.Println("\n=== 陷阱 4: 指標取址 ===")
    
    items := []string{"a", "b", "c"}
    var ptrs []*string
    
    // 錯誤：取址迴圈變數
    for _, item := range items {
        ptrs = append(ptrs, &item) // 所有指標指向同一個變數
    }
    
    fmt.Println("錯誤的指標:")
    for i, ptr := range ptrs {
        fmt.Printf("ptrs[%d] = %s\n", i, *ptr) // 都指向最後一個值
    }
    
    // 正確：取址 slice 元素
    ptrs = nil
    for i := range items {
        ptrs = append(ptrs, &items[i]) // 取址原始 slice 元素
    }
    
    fmt.Println("\n正確的指標:")
    for i, ptr := range ptrs {
        fmt.Printf("ptrs[%d] = %s\n", i, *ptr)
    }
}

## 8. switch 語句

Go 的 switch 語句比其他語言更靈活：
- 不需要 break（自動跳出）
- 支援多個值匹配
- 可以使用表達式
- 支援型態切換

### 基本 switch 用法

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "time"
)

func main() {
    fmt.Println("=== 基本 switch ===")
    
    day := "Tuesday"
    switch day {
    case "Monday":
        fmt.Println("週一，新的開始")
    case "Tuesday":
        fmt.Println("週二，繼續努力")
    case "Wednesday":
        fmt.Println("週三，過半了")
    case "Thursday", "Friday": // 多個值
        fmt.Println("快週末了")
    case "Saturday", "Sunday":
        fmt.Println("週末休息")
    default:
        fmt.Println("無效的日期")
    }
    
    fmt.Println("\n=== 帶初始化的 switch ===")
    
    switch hour := time.Now().Hour(); {
    case hour < 6:
        fmt.Println("凌晨時分")
    case hour < 12:
        fmt.Println("上午時光")
    case hour < 18:
        fmt.Println("下午時段")
    case hour < 22:
        fmt.Println("晚上時間")
    default:
        fmt.Println("深夜時分")
    }
    
    fmt.Println("\n=== 表達式 switch ===")
    
    score := 85
    switch {
    case score >= 90:
        fmt.Println("優秀")
    case score >= 80:
        fmt.Println("良好")
    case score >= 70:
        fmt.Println("普通")
    case score >= 60:
        fmt.Println("及格")
    default:
        fmt.Println("不及格")
    }
    
    fmt.Println("\n=== fallthrough 用法 ===")
    
    grade := 'B'
    switch grade {
    case 'A':
        fmt.Print("優秀，")
        fallthrough // 繼續執行下一個 case
    case 'B':
        fmt.Print("良好，")
        fallthrough
    case 'C':
        fmt.Print("普通，")
        fallthrough
    case 'D':
        fmt.Println("需要改進")
    case 'F':
        fmt.Println("不及格")
    }
}

### 型態切換 (Type Switch)

In [ ]:
//go:build ignore
package main

import "fmt"

func processValue(value interface{}) {
    switch v := value.(type) {
    case int:
        fmt.Printf("整數: %d, 平方: %d\n", v, v*v)
    case float64:
        fmt.Printf("浮點數: %.2f, 開根: %.2f\n", v, v*0.5)
    case string:
        fmt.Printf("字串: %q, 長度: %d\n", v, len(v))
    case bool:
        if v {
            fmt.Println("布林值: 真")
        } else {
            fmt.Println("布林值: 假")
        }
    case []int:
        fmt.Printf("整數切片: %v, 長度: %d\n", v, len(v))
    case map[string]int:
        fmt.Printf("字串到整數的映射: %v\n", v)
    case nil:
        fmt.Println("nil 值")
    default:
        fmt.Printf("未知型態: %T, 值: %v\n", v, v)
    }
}

func main() {
    fmt.Println("=== 型態切換範例 ===")
    
    values := []interface{}{
        42,
        3.14,
        "Hello, World!",
        true,
        []int{1, 2, 3},
        map[string]int{"a": 1, "b": 2},
        nil,
        struct{ Name string }{"Alice"},
    }
    
    for i, value := range values {
        fmt.Printf("\n[%d] ", i)
        processValue(value)
    }
    
    fmt.Println("\n=== 型態斷言 vs 型態切換 ===")
    
    var data interface{} = "hello"
    
    // 型態斷言（可能 panic）
    if str, ok := data.(string); ok {
        fmt.Printf("型態斷言成功: %s\n", str)
    }
    
    // 型態切換（安全）
    switch v := data.(type) {
    case string:
        fmt.Printf("型態切換: 字串 %s\n", v)
    case int:
        fmt.Printf("型態切換: 整數 %d\n", v)
    }
}

### switch 的進階應用

In [ ]:
//go:build ignore
package main

import (
    "fmt"
    "strings"
)

func main() {
    fmt.Println("=== 複雜條件匹配 ===")
    
    text := "Hello World"
    switch {
    case strings.HasPrefix(text, "Hello"):
        fmt.Println("以 Hello 開頭")
    case strings.HasSuffix(text, "World"):
        fmt.Println("以 World 結尾") // 不會執行，因為上面已匹配
    case strings.Contains(text, "Go"):
        fmt.Println("包含 Go")
    default:
        fmt.Println("普通字串")
    }
    
    fmt.Println("\n=== 函數返回值匹配 ===")
    
    getStatus := func() string {
        return "active"
    }
    
    switch getStatus() {
    case "active":
        fmt.Println("系統運行中")
    case "inactive":
        fmt.Println("系統已停止")
    case "maintenance":
        fmt.Println("維護中")
    }
    
    fmt.Println("\n=== 錯誤處理模式 ===")
    
    type CustomError struct {
        Code    int
        Message string
    }
    
    func (e CustomError) Error() string {
        return e.Message
    }
    
    var err error = CustomError{Code: 404, Message: "Not Found"}
    
    switch e := err.(type) {
    case CustomError:
        fmt.Printf("自定義錯誤 - 代碼: %d, 訊息: %s\n", e.Code, e.Message)
    case *CustomError:
        fmt.Printf("自定義錯誤指標 - 代碼: %d\n", e.Code)
    case nil:
        fmt.Println("沒有錯誤")
    default:
        fmt.Printf("其他錯誤: %v\n", e)
    }
    
    fmt.Println("\n=== 巢狀 switch ===")
    
    category := "electronics"
    item := "phone"
    
    switch category {
    case "electronics":
        fmt.Println("電子產品類別:")
        switch item {
        case "phone":
            fmt.Println("  -> 手機")
        case "laptop":
            fmt.Println("  -> 筆記型電腦")
        case "tablet":
            fmt.Println("  -> 平板電腦")
        default:
            fmt.Println("  -> 其他電子產品")
        }
    case "clothing":
        fmt.Println("服裝類別")
    default:
        fmt.Println("未知類別")
    }
    
    fmt.Println("\n=== 空 switch (替代 if-else) ===")
    
    x := 15
    
    // 空 switch，根據條件執行
    switch {
    case x < 0:
        fmt.Println("負數")
    case x == 0:
        fmt.Println("零")
    case x > 0 && x < 10:
        fmt.Println("小正數")
    case x >= 10 && x < 100:
        fmt.Println("中等正數")
    default:
        fmt.Println("大正數")
    }
}

## 9. goto 語句

雖然 Go 支援 `goto`，但應該謹慎使用。現代 Go 程式設計很少需要 `goto`。

### goto 的基本用法與限制

In [ ]:
//go:build ignore
package main

import "fmt"

func demonstrateGoto() {
    fmt.Println("=== goto 基本用法 ===")
    
    i := 0
    
Loop:
    if i < 3 {
        fmt.Printf("i = %d\n", i)
        i++
        goto Loop
    }
    
    fmt.Println("迴圈結束")
}

func gotoCleanup() {
    fmt.Println("\n=== goto 用於錯誤處理和清理 ===")
    
    // 模擬資源分配
    var resource1, resource2, resource3 bool
    
    // 分配資源1
    resource1 = true
    fmt.Println("分配資源1")
    
    // 模擬錯誤
    if true { // 假設這裡有錯誤
        fmt.Println("分配資源2失敗")
        goto cleanup_resource1
    }
    
    // 分配資源2（不會執行到）
    resource2 = true
    fmt.Println("分配資源2")
    
    // 模擬另一個錯誤
    if false {
        fmt.Println("分配資源3失敗")
        goto cleanup_resource2
    }
    
    resource3 = true
    fmt.Println("分配資源3")
    
    // 正常執行
    fmt.Println("所有資源分配成功，執行主要邏輯")
    
    // 清理資源3
    if resource3 {
        fmt.Println("清理資源3")
    }
    
cleanup_resource2:
    // 清理資源2
    if resource2 {
        fmt.Println("清理資源2")
    }
    
cleanup_resource1:
    // 清理資源1
    if resource1 {
        fmt.Println("清理資源1")
    }
    
    fmt.Println("函數結束")
}

func demonstrateGotoLimitations() {
    fmt.Println("\n=== goto 的限制 ===")
    
    // 不能跳過變數宣告
    if true {
        // goto skip // 編譯錯誤！不能跳過 x 的宣告
        x := 42
        fmt.Printf("x = %d\n", x)
    }
    
skip:
    fmt.Println("跳過點")
    
    // goto 不能跨函數
    // goto outsideFunction // 編譯錯誤！
}

// outsideFunction:
//     fmt.Println("不能從其他函數跳到這裡")

func betterAlternatives() {
    fmt.Println("\n=== 更好的替代方案 ===")
    
    // 用 for + break 替代 goto 迴圈
    for i := 0; i < 3; i++ {
        fmt.Printf("更好的迴圈: i = %d\n", i)
    }
    
    // 用 defer 替代 goto 清理
    func() {
        fmt.Println("\n使用 defer 進行清理:")
        
        // 資源1
        resource1 := true
        fmt.Println("分配資源1")
        defer func() {
            if resource1 {
                fmt.Println("defer 清理資源1")
            }
        }()
        
        // 資源2
        resource2 := false
        // 模擬分配失敗
        if false {
            resource2 = true
            fmt.Println("分配資源2")
            defer func() {
                fmt.Println("defer 清理資源2")
            }()
        }
        
        fmt.Println("函數結束，defer 自動清理")
    }()
}

func main() {
    demonstrateGoto()
    gotoCleanup()
    demonstrateGotoLimitations()
    betterAlternatives()
    
    fmt.Println("\n=== 總結 ===")
    fmt.Println("goto 的使用時機:")
    fmt.Println("1. 複雜的錯誤處理和資源清理")
    fmt.Println("2. 跳出多層迴圈（但標記 break 更好）")
    fmt.Println("3. 狀態機實作（但 switch 更清晰）")
    fmt.Println("\n大多數情況下，使用 for、switch、defer 等更好")
}

## 10. 選擇正確的控制結構

### 決策指南

In [ ]:
//go:build ignore
package main

import "fmt"

// 範例：不同場景下的最佳選擇
func demonstrateChoices() {
    fmt.Println("=== 控制結構選擇指南 ===")
    
    // 1. 簡單條件：使用 if
    age := 25
    if age >= 18 {
        fmt.Println("成年人")
    }
    
    // 2. 多重條件：考慮 switch vs if-else
    
    // 離散值比較：使用 switch
    grade := 'A'
    switch grade {
    case 'A', 'B':
        fmt.Println("優秀")
    case 'C':
        fmt.Println("普通")
    default:
        fmt.Println("需要改進")
    }
    
    // 範圍比較：使用 if-else 或空 switch
    score := 85
    switch {
    case score >= 90:
        fmt.Println("優秀")
    case score >= 80:
        fmt.Println("良好")
    default:
        fmt.Println("需要努力")
    }
    
    // 3. 迴圈選擇
    
    // 已知次數：使用傳統 for
    fmt.Println("\n已知次數的迴圈:")
    for i := 0; i < 3; i++ {
        fmt.Printf("迭代 %d\n", i)
    }
    
    // 條件驅動：使用 while 風格的 for
    fmt.Println("\n條件驅動的迴圈:")
    x := 1
    for x < 8 {
        fmt.Printf("x = %d\n", x)
        x *= 2
    }
    
    // 遍歷集合：使用 for-range
    fmt.Println("\n遍歷集合:")
    items := []string{"apple", "banana", "cherry"}
    for i, item := range items {
        fmt.Printf("%d: %s\n", i, item)
    }
    
    // 4. 跳出控制
    
    // 單層跳出：break/continue
    fmt.Println("\n單層跳出:")
    for i := 0; i < 5; i++ {
        if i == 2 {
            continue
        }
        if i == 4 {
            break
        }
        fmt.Printf("處理 %d\n", i)
    }
    
    // 多層跳出：標記 break
    fmt.Println("\n多層跳出:")
Outer:
    for i := 0; i < 3; i++ {
        for j := 0; j < 3; j++ {
            if i == 1 && j == 1 {
                fmt.Printf("在 (%d,%d) 跳出所有迴圈\n", i, j)
                break Outer
            }
            fmt.Printf("(%d,%d) ", i, j)
        }
        fmt.Println()
    }
}

func main() {
    demonstrateChoices()
    
    fmt.Println("\n=== 效能考量 ===")
    fmt.Println("1. switch 比多重 if-else 更高效")
    fmt.Println("2. for-range 對於集合遍歷最安全")
    fmt.Println("3. 避免不必要的 goto")
    fmt.Println("4. 提早返回減少嵌套深度")
}

## 總結

### 控制結構使用指南

| 結構 | 適用場景 | 優點 | 注意事項 |
|------|----------|------|----------|
| **if** | 簡單條件判斷 | 清晰直觀、支援初始化 | 避免深層嵌套 |
| **switch** | 多值匹配、型態判斷 | 高效、自動 break | 注意 fallthrough 用法 |
| **for (傳統)** | 已知迭代次數 | 控制精確、效能佳 | 注意邊界條件 |
| **for (while風格)** | 條件驅動迴圈 | 靈活的終止條件 | 小心無窮迴圈 |
| **for-range** | 集合遍歷 | 安全、慣用 | 小心變數重用陷阱 |
| **break/continue** | 迴圈控制 | 精確控制流程 | 配合標記處理多層 |
| **goto** | 錯誤處理、清理 | 直接跳轉 | 謹慎使用，優先考慮其他方案 |

### 最佳實務

1. **作用域管理**
   - 利用區塊限制變數作用域
   - 小心變數遮蔽問題
   - 使用初始化語句減少變數生命週期

2. **代碼可讀性**
   - 提早返回避免深層嵌套
   - 選擇合適的控制結構
   - 使用標記處理複雜的跳轉邏輯

3. **效能考量**
   - switch 優於多重 if-else
   - for-range 適合集合操作
   - 避免不必要的條件檢查

4. **錯誤避免**
   - 注意 for-range 的變數重用
   - 小心切片在迴圈中的修改
   - 合理使用 break 和 continue

掌握這些控制結構是寫出高品質 Go 程式碼的基礎。下一章將學習函式的設計與使用。